# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas
3. Ejecuta las 2 celdas de abajo para entregar

**NO requiere cuenta ni login** - solo ingresa tu nombre y listo.

In [ ]:
import json
import urllib.request
import urllib.error

# Check if we're in Pyodide (browser environment)
try:
    import pyodide
    print("✅ Pyodide detectado - Estamos en el navegador")
except ImportError:
    print("⚠️ Este notebook solo funciona en JupyterLite")
    print("   Visita: https://juakongdf.github.io/chaos-lab/")
    raise

SERVER_URL = "https://chaos-sub.vercel.app/submit"

def submit_notebook(student_name):
    """Envía el notebook actual al servidor"""
    
    if not student_name or not student_name.strip():
        print("❌ Por favor ingresa tu nombre")
        return False
    
    print("⏳ Enviando tu trabajo...")
    
    # Read the notebook file
    try:
        with open("99_Entregar_Trabajo.ipynb", 'r') as f:
            notebook_content = json.load(f)
    except Exception as e:
        print(f"❌ Error leyendo notebook: {e}")
        return False
    
    # Prepare payload
    payload = {
        "student_name": student_name.strip(),
        "notebook": notebook_content
    }
    
    # Send to server
    try:
        data = json.dumps(payload).encode('utf-8')
        req = urllib.request.Request(
            SERVER_URL,
            data=data,
            headers={'Content-Type': 'application/json'}
        )
        
        with urllib.request.urlopen(req) as response:
            result = json.loads(response.read().decode('utf-8'))
            print(f"✅ ¡Trabajo entregado exitosamente!")
            print(f"📄 Archivo: {result['filename']}")
            return True
            
    except urllib.error.HTTPError as e:
        error_data = json.loads(e.read().decode('utf-8'))
        print(f"❌ Error del servidor: {error_data.get('error', 'Error desconocido')}")
        return False
    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return False

print("✅ Sistema configurado")
print("👇 Para entregar, ejecuta la celda de abajo")

In [ ]:
# 📝 INGRESA TU NOMBRE Y EJECUTA ESTA CELDA

# Cambia "Tu Nombre Aquí" por tu nombre completo:
MI_NOMBRE = "Tu Nombre Aquí"

# Ejecuta esta celda para entregar tu trabajo
submit_notebook(MI_NOMBRE)